In [ ]:
import json, os
from pathlib import Path

from freqtrade.configuration import Configuration
from freqtrade.data.btanalysis import load_trades_from_db, load_backtest_data, load_backtest_stats
from freqtrade.data.history import load_pair_history
from freqtrade.data.dataprovider import DataProvider
from freqtrade.plugins.pairlistmanager import PairListManager
from freqtrade.exceptions import ExchangeError, OperationalException
from freqtrade.exchange import Exchange
from freqtrade.resolvers import ExchangeResolver, StrategyResolver

import numpy as np
import pandas as pd
import random
from collections import deque
import matplotlib.pyplot as plt
import time

import nest_asyncio
nest_asyncio.apply()
    
def do_analysis(pair, strategy, timeframe, data_location, sqlite_file=None, only_backtest=False, hist_and_backtest=True):
    if only_backtest is True:
        print("Loading backtest data...")
        trades = load_backtest_data(backtest_dir)
    else:
        print("Loading historic data...")
        candles = load_pair_history(datadir=data_location,
                                    timeframe=timeframe,
                                    pair=pair,
                                    data_format = "json",
                                    )

        # Confirm success
        print("Loaded " + str(len(candles)) + f" rows of data for {pair} from {data_location}")
        df = strategy.analyze_ticker(candles, {'pair': pair})
        
        if hist_and_backtest is True:
            print("Loading backtest trades data...")
            trades = load_backtest_data(backtest_dir, strat_name)
        else:
            if sqlite_file is not None:
                print("Loading DB trades data...")
                trades = load_trades_from_db(f"sqlite:///{sqlite_file}")
            else:
                # Fetch trades from database
                print("Loading DB trades data...")
                trades = load_trades_from_db(ft_config['db_url'])

        print(f"Generated {df['buy'].sum()} buy / {df['sell'].sum()} sell signals")
        data = df.set_index('date', drop=False)
        
        return (data, trades)
    
    return (pd.DataFrame(), trades)

def do_plot(pair, data, trades, d_start, d_end, plot_config=None):
    from freqtrade.plot.plotting import generate_candlestick_graph
    import plotly.offline as pyo

    trades_red = pd.DataFrame()
    
    if trades.shape[0] > 0:
        # Filter trades to one pair
        trades_red = trades.loc[trades['pair'] == pair].copy()
    
    buyf = data[data.filter(regex=r'^buy', axis=1).values==1]

    if buyf.shape[0] > 0 and trades_red.shape[0] > 0:
        for t, v in trades_red.open_date.items():
            tc = buyf.loc[(buyf['date'] < v)]
            if tc is not None and tc.shape[0] > 0:
                bt = tc.iloc[-1].filter(regex=r'^buy', axis=0)
                bt.dropna(inplace=True)
                #bt.drop("buy", inplace=True)
                trades_red.loc[t, 'sell_reason'] = f"{trades_red.loc[t, 'buy_tag']} / {trades_red.loc[t, 'sell_reason']}"
    
    # Limit graph period to your BT timerange
    data_red = data[d_start:d_end]
    
    plotconf = strategy.plot_config
    if plot_config is not None:
        plotconf = plot_config
    
    # Generate candlestick graph
    graph = generate_candlestick_graph(pair=pair,
                                       data=data_red,
                                       trades=trades_red,
                                       plot_config=plotconf
                                       )
    
    graph.update_layout(autosize=False,width=1400,height=1200)
    pyo.iplot(graph, show_link = False)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))    

In [ ]:
####################
### YOUR CONFIG HERE
####################
configs = ["config.json"]
ft_config = Configuration.from_files(files=configs)

strat_name = "YourStrategy"
stake = "USDT"
timeframe = "5m"
timeframe_detail = "1m"

## these should match your last backtested timerange
## start date
ds = '2021-11-01'
## end date
de = '2021-11-17'

## set the pairlist you want to plot
pairlist = ["ETH", "BTC"]

## set an sqlite file if you're looking at dry/live data instead of backtests
sqlite_file = None # "trades-v3.sqlite"

## set your backtest/historical data locations
backtest_dir = Path(ft_config['user_data_dir'], 'backtest_results')
data_location = Path(ft_config['user_data_dir'], 'data', ft_config['exchange']['name']')
# data_location = Path('/path', 'to', 'another', 'data', 'folder')

# example inline plot config to override one in your strat. delete if you have this config in your strategy
plot_config = {
    'main_plot': {
        'ema_100': {'color': 'black'},
    },
    'subplots': {
        "MAC": {
            'macd': {'color': 'red'},
            'macdsignal': {'color': 'black'},
        },
        "RSI": {
            'rsi': {'color': 'pink'},
        },
    }
}
                     
#########################
## DO NOT EDIT BELOW HERE                     
#########################
ft_config['strategy'] = strat_name
ft_config['timeframe'] = timeframe
ft_config['timeframe_detail'] = timeframe_detail
ft_config['datadir'] = data_location
ft_exchange = ExchangeResolver.load_exchange(ft_config['exchange']['name'], config=ft_config, validate=True)
ft_pairlists = PairListManager(ft_exchange, ft_config)
ft_dataprovider = DataProvider(ft_config, ft_exchange, ft_pairlists)

# Load strategy using values set above
strategy = StrategyResolver.load_strategy(ft_config)
strategy.dp = ft_dataprovider

if ft_config["timeframe"] is not None:
    timeframe = ft_config["timeframe"]
    print("Using config timeframe:" , timeframe)
elif strategy.timeframe is not None:
    timeframe = strategy.timeframe
    print("Using strategy timeframe:" , timeframe)
else:
    print("Using default timeframe:" , timeframe)

hist_and_backtest = True
if sqlite_file is not None:
    hist_and_backtest = False

## don't do this for more than a few pairs and for a few days otherwise Slowness Will Occur
for i in pairlist:
    (data, trades) = do_analysis(f"{i}/{stake}", strategy, timeframe, data_location, sqlite_file=sqlite_file, hist_and_backtest=hist_and_backtest)
    do_plot(f"{i}/{stake}", data, trades, ds, de, plot_config=plot_config)